### Data source: https://www.kaggle.com/anthonypino/melbourne-housing-market

In [ ]:
# Remember to change the datafile to the name you store on your own computer.
datafile = "C:\\Users\\Anat\\PythonCode\\resources\\lesson4.3_MELBOURNE_HOUSE_PRICES_LESS.csv" 

In [ ]:
import pandas as pd
import numpy as np

<h4><b>read_csv</b>: A pandas function that reads a comma separated file</h4>
read_csv will try to format the data so that it is the correct type and will report any typing problems<br>
It will also look for a header row. 
<br>http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

In [ ]:
data = pd.read_csv(datafile)
data

<h4>Let's examine our data</h4>

In [ ]:
data.info()

<h4>Looks like Unique Key really is a unique key and can serve as an index</h4>

In [ ]:
data = pd.read_csv(datafile,index_col='Address')

In [ ]:
data.iloc[1:10]

<h4>Columns 4 has mixed types</h4>

<h4>Column 4 is incident zip</h4>
Let's examine it<br>
The unique() function returns unique values in a column

In [ ]:
data['Postcode'].unique()

<h4>Some issues</h4>
<li>Sometimes zip is a float, other times it is a str
<li>Zipcodes that are represented as floats and start with 0 are missing the first digit
<li>Some zipcodes have the 4 digit extension added. Comparison becomes tough
<li>What the heck is zip 0?
<li>What about the missing (nan) values? The ? (question mark)? "UNKNOWN"?

<h2>The first step in data cleaning is to:</h2>
<h3>Decide what to do with "bad" data (missing, wrong etc.). Convert to Nan or delete the record.</h3>
<h3>Make sure all data in a column is in the correct format (convert floats to ints, get rid of unnecessary chars)</h3>
<h3>Decide what to do with missing values (NaNs)</h3>

In [ ]:
def fix_postcode(input_postcode):
    try:
        input_postcode = int(float(input_postcode))
    except:
        try:
            input_postcode = int(input_postcode.split('-')[0])
        except:
            return np.NaN
    if input_postcode < 1000 or input_postcode > 9999:
        return np.NaN
    return str(input_postcode)
        

<h4>And test it</h4>

In [ ]:
fix_postcode('3211.00')

In [ ]:
fix_postcode('1211-2')

<h3>Next, we'll apply this function to every element in input postcode to get a revised column</h3>
<h4>The pandas function "apply" applies a function to a dataframe column
<li>fix_postcode will be applied to each element of the Postcode column and we replace the existing column with the modified one

In [ ]:
data['Postcode'] = data['Postcode'].apply(fix_postcode)

In [ ]:
data['Postcode'].unique()

<h3>Finally, we'll get rid of all rows that have postcode == Nan</h3>
<li>We don't have to, that's just a choice we're making</li>

In [ ]:
data = data[data['Postcode'].notnull()]

In [ ]:
#data['Postcode'].notnull()

In [ ]:
data

<h3>Let's take a look at the columns again</h3>

In [ ]:
data.info()

<h3>Price also has missing values</h3>
<h3>Let's get rid of them</h3>

In [ ]:
data = data[(data['Price'].notnull())]

In [ ]:
data.info()

<h4>Let's look at the number of houses sold per subburb</h4>

In [ ]:
data.groupby('Suburb').count()

#### lets calculate the average price per area

In [ ]:
data[['Price','CouncilArea']].groupby('CouncilArea').mean()

In [ ]:
data2= data.copy()

In [ ]:
id(data)

In [ ]:
id(data2)

### calculate avg price at each suburb

In [ ]:
data.groupby('Suburb')['Price'].mean()

### let's add a column that contains the size of a property relative to the average size in the district :
#### we first calculate average number of rooms in the each district 
#### then we will calculate the relative size (i.e. number of rooms)

In [ ]:
data['Avg_room_num'] = data['Rooms'].groupby(data['CouncilArea']).transform('mean')

In [ ]:
data

In [ ]:
data['Rel_room_num'] = data.apply(lambda row: row.Rooms/row.Avg_room_num, axis=1)
#data['Rooms']/data['Avg_room_num']

<h4>Dealing with time</h4>
<li>Dates and times are best converted to datetime
<li>That way they will be useful for analysis because we can compute timedelta objects

In [ ]:
data.loc[1,'Date']

In [ ]:
type(data.loc[1,'Date'])

In [ ]:
import datetime
data['Date'] = data['Date'].apply(lambda x:datetime.datetime.strptime(x,'%d/%m/%Y'))


### Additional datetime components    
<li>%Y: Year (4 digits)
<li>%m: Month
<li>%d: Day of month
<li>%H: Hour (24 hour)
<li>%M: Minutes
<li>%S: Seconds
<li>%f: Microseconds
<br>
<a href = https://stackabuse.com/converting-strings-to-datetime-in-python/>For more information</a>

### Try different date formats for yourself:

In [ ]:
str1 = "2018-06-29 08:15:27.243860"
datetime.datetime.strptime(str1,'%Y-%m-%d %H:%M:%S.%f')

In [ ]:
str2 = "2019-11-01 00:00:02 UTC"
datetime.datetime.strptime(str2,'%Y-%m-%d %H:%M:%S UTC')

In [ ]:
type(data.loc[1,'Date'])

<h2>Finally, let's write a function that incorporates all our changes</h2>
## this is usefull in order to run everything together

In [ ]:
def read_real_estate_data(datafile):
    import pandas as pd
    import numpy as np
    #Add the fix_zip function
    def fix_postcode(input_postcode):
        try:
            input_postcode = int(float(input_postcode))
        except:
            try:
                input_postcode = int(input_postcode.split('-')[0])
            except:
                return np.NaN
        if input_postcode < 1000 or input_postcode > 9999:
            return np.NaN
        return str(input_postcode)
    
    #Read the file
    df = pd.read_csv(datafile)
    
    #fix the zip
    df['Postcode'] = df['Postcode'].apply(fix_postcode)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    
    df = df.dropna(how='any')    
    
    #Convert times to datetime and create a processing time column
    data['Avg_room_num'] = data.groupby('CouncilArea')['Rooms'].transform('mean')
    data['Rel_room_num'] = data.apply(lambda row: row.Rooms/row.Avg_room_num, axis=1) #data['Rooms']/data['Avg_room_num']
    
    import datetime
    df['Date'] = df['Date'].apply(lambda x:datetime.datetime.strptime(x,'%d/%m/%Y'))    
   
    return df
    

In [ ]:
df = read_real_estate_data(datafile)
df.info()

### this code creates a pivot table of the mean price for each room number and Type combination:


In [ ]:

pd.pivot_table(df, index = 'Rooms', columns = 'Type', values = 'Price', aggfunc= 'mean' )